In [1]:
import databricks.koalas as ks
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[7]").setAppName('ad eda')
sc = SparkContext(conf=conf)


In [7]:
train_user = ks.read_csv("data/train_preliminary/user.csv")

train_click_log = ks.read_csv("data/train_preliminary/click_log.csv")

train_ad = ks.read_csv("data/train_preliminary/ad.csv")


In [8]:
train_data = train_user.merge(train_click_log, on="user_id",
                              how='inner').merge(train_ad,
                                                 on="creative_id",
                                                 how='inner')

In [69]:
698970/900000


0.7766333333333333

In [9]:
test_click_log = ks.read_csv("data/test/click_log.csv")


In [10]:
# 计算广告年点击次数
sql = '''
select creative_id,
        age,
        sum(nvl(click_times, 0)) click_times
from {train_data}
group by  creative_id, age
'''
age_data = ks.sql(sql, train_data=train_data)
age_data.head()

,creative_id,age,click_times
0,148,2,4
1,148,4,5
2,148,7,3
3,148,6,1
4,148,8,1


In [16]:
# 计算广告年龄分布
sql = '''
SELECT creative_id,
       age,
       click_times,
       click_times / sum(click_times)
                         OVER (PARTITION BY creative_id  ORDER BY click_times DESC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) age_dist
FROM {age_data}
'''
age_dist_data = ks.sql(sql, age_data=age_data)
age_dist_data.head(10)
age_dist_data.cache()

,creative_id,age,click_times,age_dist
0,148,4,5,0.263158
1,148,2,4,0.210526
2,148,7,3,0.157895
3,148,3,3,0.157895
4,148,5,2,0.105263


In [19]:
sql ='''
SELECT c.user_id,
       c.age,
       c.score
FROM (
         SELECT b.user_id,
                b.age,
                b.score,
                row_number() OVER (PARTITION BY user_id ORDER BY b.score DESC) rn
         FROM (
                  SELECT a.user_id,
                         a.age,
                         sum(a.click_rate * a.age_dist) score
                  FROM (
                           SELECT a.user_id,
                                  b.age,
                                  b.age_dist,
                                  b.click_times / sum(b.click_times)
                                                      OVER (PARTITION BY user_id ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) click_rate
                           FROM {train_click_log} a
                                    LEFT JOIN {age_dist_data} b
                                              ON a.creative_id = b.creative_id
                       ) a
                  GROUP BY a.user_id,
                           a.age
              ) b
     ) c
WHERE c.rn = 1
ORDER BY c.score DESC
'''
age_dist_result = ks.sql(sql, train_click_log=train_click_log, age_data_dist=age_dist_data)
age_dist_result.head(10)

,user_id,age,score
0,248794,1,0.797569
1,806730,4,0.722222
2,279925,1,0.682429
3,348938,1,0.674038
4,271612,1,0.658615
5,459452,1,0.649389
6,870227,1,0.632195
7,636493,1,0.614476
8,817158,1,0.612521
9,551168,1,0.600752


In [22]:
age_dist_result.cache()

,user_id,age,score
0,248794,1,0.797569
1,806730,4,0.722222
2,279925,1,0.682429
3,348938,1,0.674038
4,271612,1,0.658615
5,459452,1,0.649389
6,870227,1,0.632195
7,636493,1,0.614476
8,817158,1,0.612521
9,551168,1,0.600752


In [24]:
age_dist_result.shape
age_dist_result.head(10)

,user_id,age,score
0,248794,1,0.797569
1,806730,4,0.722222
2,279925,1,0.682429
3,348938,1,0.674038
4,271612,1,0.658615
5,459452,1,0.649389
6,870227,1,0.632195
7,636493,1,0.614476
8,817158,1,0.612521
9,551168,1,0.600752


In [27]:
pred_age = train_user.merge(age_dist_result, how='inner', on=['user_id','age'])

In [28]:
pred_age.shape

(229767, 4)

In [71]:
sql = '''
SELECT a.user_id,
       a.age,
       a.age_dist
FROM (
         SELECT a.user_id,
                b.age,
                b.age_dist,
                row_number() OVER (PARTITION BY user_id ORDER BY b.age_dist DESC) rn
         FROM {test_click_log} a
                  LEFT JOIN {age_data_dist} b
                            ON a.creative_id = b.creative_id
     ) a
WHERE a.rn = 1
'''
test_age_pred_data = ks.sql(sql, test_click_log=test_click_log, age_data_dist=age_data_dist)

In [72]:
test_age_pred_data.head()



,user_id,age,age_dist
0,3000113,2,1.000000
1,3000375,2,1.000000
2,3000633,1,0.500000
3,3001030,4,0.428571
4,3001061,3,0.280815


In [73]:
test_age_pred_data.shape



(1000000, 3)

In [74]:
test_age_pred_data.cache()



,user_id,age,age_dist
0,3000113,2,1.000000
1,3000375,2,1.000000
2,3000633,1,0.500000
3,3001030,4,0.428571
4,3001061,3,0.280815
5,3001312,5,0.500000
6,3001517,4,0.500000
7,3001633,2,0.542857
8,3001877,3,1.000000
9,3001909,2,0.571429


In [75]:
test_age_pred_data.head()



,user_id,age,age_dist
0,3000113,2,1.000000
1,3000375,2,1.000000
2,3000633,1,0.500000
3,3001030,4,0.428571
4,3001061,3,0.280815


In [76]:
test_age_pred_data = test_age_pred_data.toPandas()



In [77]:
test_age_pred_data.head()



,user_id,age,age_dist
0,3000113,2,1.000000
1,3000375,2,1.000000
2,3000633,1,0.500000
3,3001030,4,0.428571
4,3001061,3,0.280815


In [78]:
test_age_pred_data = test_age_pred_data.drop('age_dist', axis=1)


In [81]:
test_age_pred_data.to_csv('./data/predict_age.csv', index=False)


In [79]:
test_age_pred_data.head()



,user_id,age
0,3000113,2
1,3000375,2
2,3000633,1
3,3001030,4
4,3001061,3


In [84]:
test_age_pred_data.describe()


,user_id,age
count,1.000000e+06,1000000.000000
mean,3.500000e+06,3.914227
std,2.886753e+05,1.953143
min,3.000001e+06,1.000000
25%,3.250001e+06,2.000000
50%,3.500000e+06,3.000000
75%,3.750000e+06,5.000000
max,4.000000e+06,10.000000


In [85]:
import pandas as pd
result = pd.read_csv("./data/result.csv")



In [86]:
result.head()



,user_id,predicted_age,predicted_gender
0,3765552,3,1
1,3096220,2,1
2,3663636,1,2
3,3516004,6,1
4,3959147,7,1


In [87]:
test_age_pred_data.columns=['user_id', 'predicted_age']
test_age_pred_data.head()



,user_id,predicted_age
0,3000113,2
1,3000375,2
2,3000633,1
3,3001030,4
4,3001061,3


In [88]:
result = result.drop('predicted_age', axis=1)
result.head()



,user_id,predicted_gender
0,3765552,1
1,3096220,1
2,3663636,2
3,3516004,1
4,3959147,1


In [89]:
result_0530 = test_age_pred_data.merge(result, on='user_id')

In [90]:
result_0530.head()



,user_id,predicted_age,predicted_gender
0,3000113,2,1
1,3000375,2,2
2,3000633,1,2
3,3001030,4,1
4,3001061,3,1


In [93]:
result_0530.isna().sum()


user_id             0
predicted_age       0
predicted_gender    0
dtype: int64

In [95]:
result_0530.to_csv('./data/result_0530.csv', index=False)
